In [1]:
import pandas as pd

DEV_META = "/home/akhil/models/DCASE24/dcase2024-task6-baseline/data/CLOTHO_v2.1/clotho_csv_files/clotho_captions_development.csv"
EVAL_META = "/home/akhil/models/DCASE24/dcase2024-task6-baseline/data/CLOTHO_v2.1/clotho_csv_files/clotho_captions_evaluation.csv"

dev_df = pd.read_csv(DEV_META)
eval_df = pd.read_csv(EVAL_META)

In [2]:
dev_df.head()

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,file_name,caption_1,caption_2,caption_3,caption_4,...,caption_6,caption_7,caption_8,caption_9,caption_10,caption_11,caption_12,caption_13,caption_14,caption_15
0,0,0,0,0,0,Distorted AM Radio noise.wav,A muddled noise of broken channel of the TV,A television blares the rhythm of a static TV.,Loud television static dips in and out of focus,The loud buzz of static constantly changes pit...,...,"Muddled noise of broken TV channel, loud telev...",A muddled noise of broken channel and the loud...,"Muddled noise of broken channel, heavy static ...",A television blares loud television static dip...,A television blares the loud buzz of static co...,"Static TV, heavy static and beginnings of a si...",Loud television static dips in and out of focu...,Loud television static dips in and out of focu...,The loud buzz of static constantly changes pit...,NaN
1,1,1,1,1,1,Paper_Parchment_Rustling.wav,A person is turning a map over and over.,A person is very carefully rapping a gift for ...,A person is very carefully wrapping a gift for...,"He sighed as he turned the pages of the book, ...",...,Carefully considering every detail.,"A person is turning something over and over, s...","A person is turning a map over and over, paper...",A person is very carefully wrapping a gift for...,"He sighed as he wrapped the gift, stopping to ...",A person is very carefully rapping a gift for ...,A person is very carefully wrapping a gift for...,Someone's breathing while papers are carefully...,"He sighed as he turned the pages of the book, ...",NaN
2,2,2,2,2,2,03 Whales Slowing Down.wav,Several barnyard animals mooing in a barn whil...,"The vocalization of several whales, along with...","Underwater, large numbers of shrimp clicking a...",Whales sing to one another over the flowing wa...,...,Several barnyard animals mooing in a barn whil...,Several barnyard animals mooing in harmony wit...,Several barnyard animals wales singing to one ...,"The vocalization of several whales, along with...","The vocalization of several whales, along with...","The vocalization of several whales, along with...","Underwater, large numbers of shrimp clicking a...","Large numbers of shrimp clicking underwater, w...",Whales sing to one another over the flowing wa...,NaN
3,3,3,3,3,3,Rope tied to boat in port.wav,An office chair is squeaking as someone bends ...,Popping and squeaking gradually tapers off to ...,Someone is opening a creaky door slowly while ...,Squeaking and popping followed by gradual popp...,...,An office chair squeaks loudly as you bend bac...,Squeaking and popping followed by gradual popp...,An office chair is squeaking as someone moves ...,Popping and squeaking gradually tapers off to ...,Popping and squeaking gradually tapers off to ...,Popping and squeaking gradually tapers off to ...,The old wooden floorboards creaked loudly as s...,"The creaky door slowly opens, while outside, a...",Squeaking and popping followed by gradual popp...,NaN
4,4,4,4,4,4,carpenter bee.wav,A flying bee is buzzing loudly around an objec...,An annoying fly is buzzing loudly and consiste...,An insect buzzing in the foreground as birds c...,"An insect trapped in a spider web struggles, b...",...,A flying bee is buzzing loudly around an objec...,A flying bee buzzes loudly around an object as...,A flying bee buzzes loudly as its wing hits so...,An annoying fly is buzzing loudly and consiste...,A loud buzz fills the air as an annoying fly t...,An annoying fly is buzzing loudly and consiste...,Insects buzzing and birds chirping with an ins...,Insect buzzing in the foreground as birds chir...,"Insect trapped in a spider web struggles, brea...",NaN


In [3]:
import sys
sys.path.insert(1, '/home/akhil/models/CLAP/src')
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

import laion_clap
model = laion_clap.CLAP_Module(enable_fusion=False)
model.load_ckpt()

/home/akhil/models/DCASE24/dcase2024-task6-baseline/aac_venv/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [4]:
from itertools import combinations
from tqdm import tqdm

consensus_scores = []
for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
    captions_list = [row['caption_1'],row['caption_2'], row['caption_3'], row['caption_4'], row['caption_5']]

    consensus = 0
    combos = 0
    # create combinations
    for combo in combinations(captions_list, 2):  # 2 for pairs, 3 for triplets, etc
        text_embed = model.get_text_embedding(combo)
        sim_score = torch.nn.functional.cosine_similarity(torch.tensor(text_embed[0]), torch.tensor(text_embed[1]), dim=-1)
        consensus += sim_score
        combos += 1

    score = consensus/combos
    consensus_scores.append(score)

  7%|▋         | 251/3839 [00:21<05:03, 11.83it/s]


KeyboardInterrupt: 

In [ ]:
consensus_scores = [x.numpy() for x in consensus_scores]
df2 = dev_df.assign(consensus_scores=consensus_scores)
df2

In [ ]:
DEV_SAVE_META = "/home/akhil/models/DCASE24/dcase2024-task6-baseline/data/CLOTHO_v2.1/clotho_csv_files/clotho_captions_development_with_consensus.csv"
df2.to_csv()

In [ ]:
df2

In [ ]:
# overall consensus
DEV_SAVE_META = "/home/akhil/models/DCASE24/dcase2024-task6-baseline/data/CLOTHO_v2.1/clotho_csv_files/clotho_captions_development_with_consensus.csv"
df2 = pd.read_csv(DEV_SAVE_META)
df2["consensus_scores"].mean()